In [1]:
import os
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-03-07 14:52:11--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.02MB/s    in 0.2s    

2022-03-07 14:52:12 (5.02 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
#pyspark imports
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.sql.functions import col, count
from pyspark.sql.types import DateType

In [4]:
# Start Spark session
spark = SparkSession.builder.appName("amazonApparelReviews").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
# Load in data
url = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz'
spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Apparel_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True, timestampFormat="yyyy/MM/dd HH:mm:ss")
df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...|         Apparel|          5|    

In [6]:
#Count the number of records (rows) in the dataset.
df.count()

5906333

In [7]:
df = df.dropna()
df = df.dropDuplicates()
df.count()

5905269

In [8]:
#Transform the dataset to fit the tables in the schema file. Be sure the DataFrames match in data type and in column name.
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
#review table
review_df = df.select(['review_id','customer_id', 'product_id','product_parent', 'review_date'])
review_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1000LG8H70CNQ|     696306|B00O3A0RHY|     246430568| 2015-01-25|
|R100BCEUAIHZQC|   11807812|B00CASTERA|     600575877| 2014-06-03|
|R100EOB2GC5UYQ|   15017601|B002MLE36W|     988639002| 2014-10-26|
|R100MYSZIXWX1D|   44291413|B007HSEDVK|        174345| 2012-10-31|
|R100UMGHMFVXKC|   24539177|B00F52CUCE|     162326772| 2014-03-18|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [ ]:
#review date as date
review_df = review_df.withColumn("review_date",col("review_date").cast(DateType()))
review_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [9]:
#product table
product_df = df.select(['product_id','product_title']).distinct()
product_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B01427DDXM|Zoozie LA Women's...|
|B013TPITFA|Tabeez Women's Pl...|
|B013FU4Y9E|Ekouaer Women's U...|
|B013FNK6PC|Ezcosplay Women O...|
|B013FACXVK|Womens Two Piece ...|
+----------+--------------------+
only showing top 5 rows



In [10]:
#customer table
customer_df = df.groupBy(['customer_id']).count()
customer_df = customer_df.withColumnRenamed('count','customer_count')
customer_df = customer_df.distinct()
customer_df.show(5)
customer_df.printSchema()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   12955905|             5|
|    3909801|             1|
|   17063052|             1|
|   13245107|             1|
|   15149043|             5|
+-----------+--------------+
only showing top 5 rows

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [ ]:
#vine table
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine'])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1000LG8H70CNQ|          5|            1|          1|   N|
|R100BCEUAIHZQC|          5|            0|          0|   N|
|R100EOB2GC5UYQ|          4|            0|          0|   N|
|R100MYSZIXWX1D|          2|            0|          0|   N|
|R100UMGHMFVXKC|          5|            0|          1|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [11]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://<endpoint>:5432/<db_instance"
config = {"user":"<username>",
          "password": "<password>",
          "driver":"org.postgresql.Driver"}

In [ ]:
 # Write DataFrames to tables
review_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
product_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
customer_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)